## IOT project ISEN ITII
## Performed by Arnaud SOUQUIERE, Claude REVERSAT & Frederic CHAIRAT


This project is a trial using IoT-LAB M3 boards withs sensors, leds, LoraWan communications & interface with Cloud.

We will try to measure on lots of boards, for example on Saclay site, temperature, atmospheric pressure & ambiant luminosity.
Then after all led turned ON, check if ambiant luminosity had increased on one specific board.

All thoses data will be transferred on The Thing Network to be put under best view

This notebook is adapted to pilote the 50 boards to turn on the Leds

### Please first submit an experiment on IoT-LAB

1. Choose your site (grenoble|lille|saclay|strasbourg):

Grenoble is recommended due to boards location situated under floor
should be the same site as Projet-IOT

In [9]:
%env SITE1=grenoble

env: SITE1=grenoble
env: SITE2=saclay


2. Submit a first experiment on 50 boards, using the following command (during 2 hours, recommended to be done on evening or night):

In [11]:
!iotlab-experiment submit -n "riot-sensors" -d 120 -l 1,archi=m3:at86rf231+site=$SITE1

Using custom api_url: https://www.iot-lab.info/api/
{
    "id": 302879
}


3. Wait for the experiment to be in the Running state:

In [13]:
!iotlab-experiment wait --timeout 30 --cancel-on-timeout --id $ID1

Using custom api_url: https://www.iot-lab.info/api/
Waiting that experiment 302879 gets in state Running
"Running"
Using custom api_url: https://www.iot-lab.info/api/
usage: iotlab-experiment wait [-h] [-i EXPERIMENT_ID] [--state STATE]
                              [--step STEP] [--timeout TIMEOUT]
                              [--cancel-on-timeout]
iotlab-experiment wait: error: argument -i/--id: invalid int value: 'xxxxxx'


**Note:** If the command above returns the message `Timeout reached, cancelling experiment <exp_id>`, try to re-submit your experiment later or try on another site.

4. Get the experiment nodes list:

Add the good ID number at --id

In [14]:
!iotlab-experiment --jmespath="items[*].network_address | sort(@)" get --nodes --id $ID1

Using custom api_url: https://www.iot-lab.info/api/
[
    "m3-101.grenoble.iot-lab.info"
]


Main & Makefile had been adapted to perform good functionallity of the programm.

7. Build and flash the application:

In [16]:
!make IOTLAB_NODE=auto flash

Using custom api_url: https://www.iot-lab.info/api/
Using custom api_url: https://www.iot-lab.info/api/
Using custom api_url: https://www.iot-lab.info/api/
Building application "Projet" for "iotlab-m3" with MCU "stm32".

"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/boards/iotlab-m3
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/boards/common/iotlab
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/core
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/cpu/stm32
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/cpu/cortexm_common
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/cpu/cortexm_common/periph
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/cpu/stm32/periph
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/cpu/stm32/stmclk
"make" -C /home/jovyan/work/training/iot-lab-training/riot/RIOT/cpu/stm32/vectors
"make" -C /home/jovyan/work/training/iot-lab-training

8. Open a Jupyter terminal (use `File > New > Terminal`) and connect to the serial port of the IoT-LAB M3 and observe the value displayed every 2 seconds:

9. In the RIOT shell, verify that the `lps` command is working as expected:

You can keep this terminal open while performing the next steps of this notebook.

### Read the lsm303dlhc sensor


To read the lsm303dlhc sensor in a loop and to start/stop this continous read from a shell command, we will use 2 things:
- the continuous read will be done in a separate thread `lsm303dlhc_thread`,
- the start/stop feature will use one of the synchronization mechanisms available in RIOT: a mutex. The idea is make the lsm303dlhc thread try to aquire the mutex before reading the values. If the mutex is already aquired by the main thread, the lsm303dlhc thread will be blocked, trying to aquire it. So the start subcommand of the `lsm` shell will just release the mutex to let the lsm303dlhc thread read the values. After each read, the lsm303dlhc thread will release the mutex. The stop subcommand of the `lsm` shell will just aquire the mutex again to clock the lsm303dlhc thread.
To read the lsm303dlhc sensor, use the lsm303dlhc module driver. In this part of the exercise, the driver is read from a separate background thread.

1. In the application [Makefile](Makefile), add the lsm303dlhc module to the build:

```Makefile
USEMODULE += lsm303dlhc
```

2. In the [main.c](main.c), add the necessary includes for this driver:

```c
#include "lsm303dlhc.h"
#include "lsm303dlhc_params.h"
```

3. Declare the device descriptor variable:

```c
static lsm303dlhc_t lsm303dlhc;
```

4. Declate the mutex used to lock/unlock the lsm303dlhc thread. The mutex is initialized locked in order to block the lsm303dlhc thread at startup.

```c
static mutex_t lsm_lock = MUTEX_INIT_LOCKED;
```


5. In the main function, initialize the driver:

```c
    lsm303dlhc_init(&lsm303dlhc, &lsm303dlhc_params[0]);
```

6. In the lsm303dlhc thread function, acquire the mutex. This call blocks the thread until the mutew is released by another thread:

```c
        mutex_lock(&lsm_lock);
```

7. In the lsm303dlhc thread function, after the mutex is aquired, the accelerometer/magnetometer values can be read:

```c
        lsm303dlhc_3d_data_t mag_value;
        lsm303dlhc_3d_data_t acc_value;
        lsm303dlhc_read_acc(&lsm303dlhc, &acc_value);
        printf("Accelerometer x: %i y: %i z: %i\n",
               acc_value.x_axis, acc_value.y_axis, acc_value.z_axis);
        lsm303dlhc_read_mag(&lsm303dlhc, &mag_value);
        printf("Magnetometer x: %i y: %i z: %i\n",
               mag_value.x_axis, mag_value.y_axis, mag_value.z_axis);
```

8. Once the values are read, the mutex must be released:

```c
        mutex_unlock(&lsm_lock);
```

9. Finally we can implement the `start`/`stop` subcommands of the `lsm` shell command `lsm303dlhc_handler` handler:

```c
    if (!strcmp(argv[1], "start")) {
        mutex_unlock(&lsm_lock);
    }
    else if (!strcmp(argv[1], "stop")) {
        mutex_trylock(&lsm_lock);
    }
    else {
        _lsm303dlhc_usage(argv[0]);
        return -1;
    }
```
   The `stop` subcommand calls the `mutex_trylock` instead of `mutex_lock` to avoid having the shell blocked when `lsm stop` is called several times in row.

10. Let's build and flash again the application:

In [ ]:
!make IOTLAB_NODE=auto flash

11. If you kept the previous serial output opened in the terminal, you can test the new `lsm` with the `start` and `stop` subcommands

### Read other sensors

As said at the beginning of this Notebook, the IoT-LAB M3 boards provides 2 other sensors:
- l3g4200d: a gyroscope
- isl29020: a light sensor

In this section, we propose that you extend the application with 2 new shell commands, `isl` and `l3g`. The synopsis of these commands is the following:
- `isl` should directly read the isl29020 light sensor and print the value:

- Similar to the lsm303dlhc, `l3g` should have the `start` and `stop` subcommands to start/stop reading the gyroscope values in a loop. The values will be read in a new thread (`l3g4200d_thread`) and the synchronization between the main thread and the l3g4200d thread will use a new mutex `l3g_lock`.

**Note:** In Grenoble, all sensors are located under an access floor, so completely in the dark. On this IoT-LAB site, the light sensors won't return usable values (always 0).

To help you with the APIs usage of the corresponding drivers, you can have a look at the following resources:
- [isl29020 API online documentation](http://doc.riot-os.org/group__drivers__isl29020.html)
- The [isl2920 test application code](https://github.com/RIOT-OS/RIOT/tree/master/tests/driver_isl29020) can also be useful
- [l3g4200d API online documentation](http://doc.riot-os.org/group__drivers__l3g4200d.html)
- The [l3g4200d test application code](https://github.com/RIOT-OS/RIOT/tree/master/tests/driver_l3g4200d) can also be useful

Also, don't forget to add the corresponding modules in the [Makefile](Makefile).

Once done, you can rebuild and flash your application.

In [ ]:
!make IOTLAB_NODE=auto flash

**Note:** If your experiment is already stopped, you can submit another one as explained in the "Submit an experiment on IoT-LAB" section.


**Optional improvement:** the continuous read is always done every 500ms. It is possible to extend the `lsm start` and `l3g start` commands with an extra parameter to specify the delay between each read:

If unspecified (`argc < 3`) the delay value is set by default to 500ms. If specified (`argc == 3`), the argument (`argv[2]`) can be converted to an int using the `atoi` function. The delay value can be stored in a global static variable (`uint32_t`).

The following command should read the lsm303dlhc values every 2s:

### Free up the resources

Since you finished the training, stop your experiment to free up the experiment nodes:

In [7]:
!iotlab-experiment stop --id $ID1

Using custom api_url: https://www.iot-lab.info/api/
{
    "id": 302848,
    "status": "Delete request registered"
}


The serial link connection through SSH will be closed automatically.